导入模块和定义变量

In [20]:
import csv
import os
import time

import numpy as np
from easydict import EasyDict as edict
from matplotlib import pyplot as plt
import mindspore
from mindspore import nn
from mindspore import context
from mindspore import dataset
from mindspore.train.callback import TimeMonitor, LossMonitor
from mindspore import Tensor
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

context.set_context(mode=context.GRAPH_MODE, device_target="CPU")
# 设定运行模式为静态图模式，并且运行设备为昇腾芯片

# 变量定义
cfg = edict({
    'data_size': 150,
    'train_size': 120,      # 训练集大小
    'test_size': 30 ,       # 测试集大小
    'feature_number': 4,       # 输入特征数
    'num_class': 3,     # 分类类别
    'batch_size': 30,
    'data_dir':    './iris.data',                     
    'save_checkpoint_steps': 5,                 # 多少步保存一次模型
    'keep_checkpoint_max': 1,                      # 最多保存多少个模型
    'out_dir_no_opt':   './model_iris/no_opt',          # 保存模型路径，无优化器模型
    'out_dir_sgd':   './model_iris/sgd',          # 保存模型路径,SGD优化器模型
    'out_dir_momentum':   './model_iris/momentum',          # 保存模型路径，momentum模型
    'out_dir_adam':   './model_iris/adam',          # 保存模型路径，adam优化器模型
    'output_prefix': "checkpoint_fashion_forward"     # 保存模型文件名
})



 读取数据并预处理

In [21]:
# 读取鸢尾花数据集，本数据集共有150个带标签的数据
with open(cfg.data_dir) as csv_file:
    data = list(csv.reader(csv_file, delimiter=','))

# 共150条数据，将数据集的4个属性作为自变量X。将数据集的3个类别映射为{0, 1，2}，作为因变量Y
label_map = {'Iris-setosa': 0,'Iris-versicolor': 1,'Iris-virginica':2 }
# 分别获取数据中的特征值X和标签值Y
X = np.array([[float(x) for x in s[:-1]] for s in data[:cfg.data_size]], np.float32)
Y = np.array([label_map[s[-1]] for s in data[:cfg.data_size]], np.int32)

# 将数据集分为训练集120条，测试集30条。
train_idx = np.random.choice(cfg.data_size, cfg.train_size, replace=False)
test_idx = np.array(list(set(range(cfg.data_size)) - set(train_idx)))
X_train, Y_train = X[train_idx], Y[train_idx]
X_test, Y_test = X[test_idx], Y[test_idx]

# 使用MindSpore GeneratorDataset接口将numpy.ndarray类型的数据转换为Dataset
def gen_data(X_train, Y_train, epoch_size):
    # 生成训练集
    XY_train = list(zip(X_train, Y_train))
    ds_train = dataset.GeneratorDataset(XY_train, ['x', 'y'])
    # 设定数据集大小
    # 打乱操作并设定batchsize
    ds_train = ds_train.shuffle(buffer_size=cfg.train_size).batch(cfg.batch_size, drop_remainder=True)
    # 生成测试集
    XY_test = list(zip(X_test, Y_test))
    ds_test = dataset.GeneratorDataset(XY_test, ['x', 'y'])
    # 设定数据集大小
    # 打乱操作并设定batchsize
    ds_test = ds_test.shuffle(buffer_size=cfg.test_size).batch(cfg.test_size, drop_remainder=True)
    return ds_train, ds_test  


定义训练、评估、测试函数

In [22]:
# 训练函数
def train(network, net_opt, ds_train, prefix, directory, print_times):
    # 定义网络损失函数
    net_loss = nn.SoftmaxCrossEntropyWithLogits(sparse=True, reduction="mean")
    # 定义模型
    model = Model(network, loss_fn=net_loss, optimizer=net_opt, metrics={"acc"})
    # 定义损失值指标
    loss_cb = LossMonitor(per_print_times=int(cfg.train_size / cfg.batch_size))
    # 设置checkpoint
    config_ck = CheckpointConfig(save_checkpoint_steps=cfg.save_checkpoint_steps,
                                 keep_checkpoint_max=cfg.keep_checkpoint_max)
    ckpoint_cb = ModelCheckpoint(prefix=prefix, directory=directory, config=config_ck)
    print("============== Starting Training ==============")
    # 训练模型
    model.train(epoch_size, ds_train, callbacks=[ckpoint_cb, loss_cb], dataset_sink_mode=False)
    return model

class_names=['setosa', 'versicolor', 'virginica']
# 评估预测函数
def eval_predict(model, ds_test):
    # 使用测试集评估模型，打印总体准确率
    metric = model.eval(ds_test)
    print(metric)
    # 预测
    test_ = ds_test.create_dict_iterator().__next__()
    test = Tensor(test_['x'], mindspore.float32)
    predictions = model.predict(test)
    predictions = predictions.asnumpy()
    true_label = test_['y'].asnumpy()
    for i in range(10):
        p_np = predictions[i, :]
        pre_label = np.argmax(p_np)
        print('第' + str(i) + '个sample预测结果：', class_names[pre_label], '   真实结果：', class_names[true_label[i]]) 


无优化器模型训练并预测

In [23]:
epoch_size = 20   # 训练20次
print('------------------无优化器--------------------------')
# 数据
ds_train, ds_test = gen_data(X_train, Y_train, epoch_size)  # 生成训练集和测试集
# 定义网络并训练
network = nn.Dense(cfg.feature_number, cfg.num_class)  # 定义一个全连接网络层，输入特征为4，输出类别为3
model = train(network, None, ds_train, "checkpoint_no_opt", cfg.out_dir_no_opt, print_times=4)  # 用训练集训练网络，设置网络结构，模型名称，保存路径, print_times
# 评估预测
eval_predict(model, ds_test)  # 用测试集进行预测 

------------------无优化器--------------------------
============== Starting Training ==============
epoch: 1 step: 4, loss is 1.0890177488327026
epoch: 2 step: 4, loss is 1.108185887336731
epoch: 3 step: 4, loss is 1.120065689086914
epoch: 4 step: 4, loss is 1.0739997625350952
epoch: 5 step: 4, loss is 1.0813523530960083
epoch: 6 step: 4, loss is 1.0865776538848877
epoch: 7 step: 4, loss is 1.1399807929992676
epoch: 8 step: 4, loss is 1.1247919797897339
epoch: 9 step: 4, loss is 1.0700039863586426
epoch: 10 step: 4, loss is 1.0894412994384766
epoch: 11 step: 4, loss is 1.0696682929992676
epoch: 12 step: 4, loss is 1.0715069770812988
epoch: 13 step: 4, loss is 1.0993870496749878
epoch: 14 step: 4, loss is 1.0425852537155151
epoch: 15 step: 4, loss is 1.099208116531372
epoch: 16 step: 4, loss is 1.1019916534423828
epoch: 17 step: 4, loss is 1.0864428281784058
epoch: 18 step: 4, loss is 1.1019978523254395
epoch: 19 step: 4, loss is 1.1108100414276123
epoch: 20 step: 4, loss is 1.088660717010

SGD优化器模型训练并预测

In [24]:
# ---------------------------------------------------SGD-------------------------------------
epoch_size = 300  # 训练300次
lr = 0.01
print('-------------------SGD优化器-----------------------')
# 数据
ds_train, ds_test = gen_data(X_train, Y_train, epoch_size) # 生成训练集和测试集
# 定义网络并训练、测试、预测
network = nn.Dense(cfg.feature_number, cfg.num_class)  # 定义一个全连接网络层，输入特征为4，输出类别为3
net_opt = nn.SGD(network.trainable_params(), lr)  # 用SGD优化器进行优化 
model = train(network, net_opt, ds_train, "checkpoint_sgd", cfg.out_dir_sgd, 40)  # 用训练集训练网络，设置网络结构，优化器，模型名称，保存路径, print_times
# 评估预测
eval_predict(model, ds_test) # 用测试集进行预测

-------------------SGD优化器-----------------------
============== Starting Training ==============
epoch: 1 step: 4, loss is 1.0794405937194824
epoch: 2 step: 4, loss is 1.0588743686676025
epoch: 3 step: 4, loss is 1.0042811632156372
epoch: 4 step: 4, loss is 1.0079346895217896
epoch: 5 step: 4, loss is 0.9641113877296448
epoch: 6 step: 4, loss is 0.945376455783844
epoch: 7 step: 4, loss is 0.943703830242157
epoch: 8 step: 4, loss is 0.8927279114723206
epoch: 9 step: 4, loss is 0.8863064050674438
epoch: 10 step: 4, loss is 0.8643203377723694
epoch: 11 step: 4, loss is 0.8530686497688293
epoch: 12 step: 4, loss is 0.8531051278114319
epoch: 13 step: 4, loss is 0.8509120345115662
epoch: 14 step: 4, loss is 0.8578325510025024
epoch: 15 step: 4, loss is 0.7968479990959167
epoch: 16 step: 4, loss is 0.7877439260482788
epoch: 17 step: 4, loss is 0.777549147605896
epoch: 18 step: 4, loss is 0.7513393759727478
epoch: 19 step: 4, loss is 0.7699323296546936
epoch: 20 step: 4, loss is 0.761829197406

Momentum优化器模型训练并预测

In [25]:
epoch_size = 20  # 训练20次
lr = 0.01  # 学习率为0.01
print('-------------------Momentum优化器-----------------------')
# 数据
ds_train, ds_test = gen_data(X_train, Y_train, epoch_size)  # 生成训练集和测试集
# 定义网络并训练
network = nn.Dense(cfg.feature_number, cfg.num_class) # 定义一个全连接网络层，输入特征为4，输出类别为3
net_opt = nn.Momentum(network.trainable_params(), lr, 0.9)   # 用 momentum 优化器进行优化，学习率为0.01，动量大小为0.9
model = train(network, net_opt, ds_train, "checkpoint_momentum", cfg.out_dir_momentum, 4)  # 用训练集训练网络，设置网络结构，优化器，模型名称，保存路径, print_times
# 评估预测
eval_predict(model, ds_test)  # 用测试集进行预测  

-------------------Momentum优化器-----------------------
============== Starting Training ==============
epoch: 1 step: 4, loss is 1.0351159572601318
epoch: 2 step: 4, loss is 0.9739065170288086
epoch: 3 step: 4, loss is 0.997841477394104
epoch: 4 step: 4, loss is 0.665257453918457
epoch: 5 step: 4, loss is 0.6971848607063293
epoch: 6 step: 4, loss is 0.6415892243385315
epoch: 7 step: 4, loss is 0.5625832080841064
epoch: 8 step: 4, loss is 0.5181046724319458
epoch: 9 step: 4, loss is 0.497139573097229
epoch: 10 step: 4, loss is 0.4468664228916168
epoch: 11 step: 4, loss is 0.49530303478240967
epoch: 12 step: 4, loss is 0.4412528872489929
epoch: 13 step: 4, loss is 0.4025491178035736
epoch: 14 step: 4, loss is 0.4438770115375519
epoch: 15 step: 4, loss is 0.38906240463256836
epoch: 16 step: 4, loss is 0.43213075399398804
epoch: 17 step: 4, loss is 0.4019095301628113
epoch: 18 step: 4, loss is 0.2954254746437073
epoch: 19 step: 4, loss is 0.38437923789024353
epoch: 20 step: 4, loss is 0.332

Adam优化器模型训练并预测

In [26]:
epoch_size = 15  # 训练15次
lr = 0.1  # 学习率为0.1, 动态学习率
print('------------------Adam优化器--------------------------')
# 数据
ds_train, ds_test = gen_data(X_train, Y_train, epoch_size)  # 生成训练集和测试集
# 定义网络并训练
network = nn.Dense(cfg.feature_number, cfg.num_class)  # 定义一个全连接网络层，输入特征为4，输出类别为3
net_opt = nn.Adam(network.trainable_params(), learning_rate=lr)  # 用 Adam 优化器进行优化，学习率为0.1
model = train(network, net_opt, ds_train, "checkpoint_adam", cfg.out_dir_adam, 4)  # 用训练集训练网络，设置网络结构，优化器，模型名称，保存路径, print_times
# 评估预测
eval_predict(model, ds_test) 

------------------Adam优化器--------------------------
============== Starting Training ==============
epoch: 1 step: 4, loss is 0.8355636596679688
epoch: 2 step: 4, loss is 0.9874771237373352
epoch: 3 step: 4, loss is 1.1438740491867065
epoch: 4 step: 4, loss is 0.6680456399917603
epoch: 5 step: 4, loss is 0.4415942430496216
epoch: 6 step: 4, loss is 0.9772436618804932
epoch: 7 step: 4, loss is 0.4867591857910156
epoch: 8 step: 4, loss is 0.4729108512401581
epoch: 9 step: 4, loss is 0.7108497023582458
epoch: 10 step: 4, loss is 0.409839004278183
epoch: 11 step: 4, loss is 0.41292107105255127
epoch: 12 step: 4, loss is 0.4746319055557251
epoch: 13 step: 4, loss is 0.2644176781177521
epoch: 14 step: 4, loss is 0.3061882555484772
epoch: 15 step: 4, loss is 0.2752925455570221
{'acc': 1.0}
第0个sample预测结果： versicolor    真实结果： versicolor
第1个sample预测结果： virginica    真实结果： virginica
第2个sample预测结果： setosa    真实结果： setosa
第3个sample预测结果： versicolor    真实结果： versicolor
第4个sample预测结果： virginica    真实结果

查看保存的模型

In [27]:
os.listdir('./model_iris/adam') # 查看保存的模型

['checkpoint_adam-graph.meta', 'checkpoint_adam-15_4.ckpt']